# Movie Recommendation System 
    Using K-means and KNN algorithm



In [54]:
import numpy as np  



from math import sqrt 

import pandas as pd 
import ast 
import warnings
warnings.filterwarnings('ignore')


1) Selecting movie dataset
2) Selecting the features  

In [55]:
myMovieData = pd.read_csv('tmdb_5000_movies.csv')
myMovieData = myMovieData[['budget','genres','revenue','title','runtime','vote_average']]
myMovieData.head()

,budget,genres,revenue,title,runtime,vote_average
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2787965087,Avatar,162.0,7.2
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",961000000,Pirates of the Caribbean: At World's End,169.0,6.9
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",880674609,Spectre,148.0,6.3
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",1084939099,The Dark Knight Rises,165.0,7.6
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",284139100,John Carter,132.0,6.1


Checking if there are any null values and if there are drop them    

In [56]:
myMovieData.isnull().sum()
myMovieData.dropna(inplace=True)

checking if there are any duplicated values and if there are drop them 

In [57]:

myMovieData.duplicated().sum()
myMovieData = myMovieData.drop_duplicates()

In [58]:

myMovieData.head()

,budget,genres,revenue,title,runtime,vote_average
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2787965087,Avatar,162.0,7.2
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",961000000,Pirates of the Caribbean: At World's End,169.0,6.9
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",880674609,Spectre,148.0,6.3
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",1084939099,The Dark Knight Rises,165.0,7.6
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",284139100,John Carter,132.0,6.1


Finds the values with name 

In [59]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L  

In [60]:
myMovieData['genres'] = myMovieData['genres'].apply(convert)


In [61]:
myMovieData['genres'] = myMovieData['genres'].str[0]


converts strings to values 

In [62]:
def stringToNumber(value):   

    if value == 'action':   
        return 9
    elif value == 'comedy':   
        return 7
    elif value == 'romance':   
        return 5
    elif value == 'adventure':
        return 3
    elif value == 'horror':
        return 1
    else:
        return 0

In [63]:
myMovieData.head()
myMovieData = myMovieData[(myMovieData[['budget','genres','budget','runtime','vote_average']] != 0).all(axis=1)]
testDataSet=myMovieData.sample(n=100)

Converts the genres into a string and remove all the values that does not come under the genres we choose

In [64]:
testDataSet['genres'].astype(str)
testDataSet['genres']=testDataSet['genres'].apply(lambda x:x.lower())
testDataSet['genres']=testDataSet['genres'].apply(stringToNumber)
testDataSet = testDataSet[(testDataSet[['genres']] != 0).all(axis=1)]

testDataSet.head()


,budget,genres,revenue,title,runtime,vote_average
140,150000000,9,205366737,White House Down,131.0,6.4
3277,7200000,7,0,Made in Dagenham,113.0,6.6
62,180000000,9,356743061,The Legend of Tarzan,109.0,5.5
1628,30000000,9,108609310,Sanctum,108.0,5.8
1964,41300000,7,106371651,The Whole Nine Yards,98.0,6.2


In [65]:
Xtrain_dataset = testDataSet[(testDataSet[['genres','budget','revenue','runtime','vote_average']] != 0).all(axis=1)]
Xtrain_dataset = testDataSet[['genres','budget','revenue','runtime','vote_average']]
tempDataset = Xtrain_dataset




Converts the values to be in between the range 1 to 10 

In [66]:
Xtrain_dataset = ((Xtrain_dataset - Xtrain_dataset.min())/(Xtrain_dataset.max() - Xtrain_dataset.min()))*9 +1

testDataSet[['budget','genres','revenue','runtime','vote_average']] = Xtrain_dataset[['budget','genres','revenue','runtime','vote_average']]
Xtrain_dataset.head()



,genres,budget,revenue,runtime,vote_average
140,10.00,7.738693,3.349626,6.4000,6.896552
3277,7.75,1.280402,1.000000,4.3750,7.517241
62,10.00,9.095477,5.081541,3.9250,4.103448
1628,10.00,2.311558,2.242613,3.8125,5.034483
1964,7.75,2.822613,2.217011,2.6875,6.275862


# Eucledian Distance 

In [67]:
def eucledian_distance(itemRow,dataRow):
    # in the begninnig the distance is 0
    distance = 0 

    # Traverse through the entire row of the dataset and the itemRow then calculate eucledian_distance 
    for column in range(len(itemRow)-1):
        distance = distance + (itemRow[column] - dataRow[column])**2

    distance = sqrt(distance)     
    return distance

# KNN algorithm

In [68]:
def getKNearestNeighbour(trainValues, unknowValue,totalNeighbours):
    saveDistance =[]
    for row in range(len(trainValues)):
        values = trainValues.iloc[row].values
        distance = eucledian_distance(unknowValue,values)
        saveDistance.append((values,distance))
      
    saveDistance.sort(key=lambda index:index[1])

    neighborList = []
    for i in range(totalNeighbours):
        neighborList.append(saveDistance[i][0])  
    return neighborList
	
		
	

	    

In [69]:
columns = Xtrain_dataset.shape[1]
rows = Xtrain_dataset.shape[0]


Calculating Centroid distances

In [70]:
def distanceFunction(oldcentorids,newCentroid):
    return  np.linalg.norm(oldcentorids - newCentroid)

In [71]:
dataNumpy = Xtrain_dataset.to_numpy()

# K means

In [72]:
for i in range(1):
    
    random = np.random.default_rng()
    totalCluster = 5
    print('cluster',totalCluster)
    totalIterations = 0

    centroids = np.zeros((totalCluster, columns))
    for i in range(totalCluster):
        random_value = int(rows * random.random())
        centroids[i] = dataNumpy[random_value]
    print('initial centroids',centroids)

    oldcentorids = np.zeros((totalCluster, columns))
    while distanceFunction(oldcentorids,centroids) > 0:
        predicedCluster = np.zeros(rows)
        for i in range(rows):
            distances = [distanceFunction(dataNumpy[i],value) for value in centroids]
            nearest_center = np.argmin(distances)
            predicedCluster[i] = nearest_center
         

   
        oldcentorids = centroids.copy()
        
        centroids = np.zeros((totalCluster,columns))
        for i in range(totalCluster):
            centroids[i] = np.average(dataNumpy[predicedCluster == i], axis=0)
        totalIterations = totalIterations+1
        
    print(totalIterations,'iterations')
print(predicedCluster)

cluster 5
initial centroids [[ 7.75        1.85929648  2.38967577  4.375       8.75862069]
 [ 7.75        2.58291457  1.95145223  3.475       3.79310345]
 [ 3.25       10.          8.13849829  5.6125      7.51724138]
 [ 3.25        2.22110553  1.36150416  7.3         7.20689655]
 [ 7.75        2.53768844  1.40413815  3.1375      8.44827586]]
4 iterations
[0. 4. 2. 1. 4. 3. 4. 4. 0. 1. 1. 1. 1. 2. 4. 2. 3. 1. 3. 4. 1. 1. 0. 1.
 1. 1. 3. 4. 3. 0. 2. 0. 1. 1. 3. 4. 2. 1. 4. 1. 0. 1. 1. 4. 4. 1. 2. 4.
 4. 1. 1. 3. 0. 0.]


In [73]:
Labels = predicedCluster
testDataSet['labels'] = Labels
testDataSet.head()


,budget,genres,revenue,title,runtime,vote_average,labels
140,7.738693,10.00,3.349626,White House Down,6.4000,6.896552,0.0
3277,1.280402,7.75,1.000000,Made in Dagenham,4.3750,7.517241,4.0
62,9.095477,10.00,5.081541,The Legend of Tarzan,3.9250,4.103448,2.0
1628,2.311558,10.00,2.242613,Sanctum,3.8125,5.034483,1.0
1964,2.822613,7.75,2.217011,The Whole Nine Yards,2.6875,6.275862,4.0


Takes the value from the user about the new movie that user would like to add.

In [74]:

usreMovie_recommended = pd.DataFrame()


add the given values to the dataset

- Gets the result of the nearest neighbours, and find 4 neighbors closest to the movie recommended. 
- Labels the new movie that has been entered by the user. 

In [75]:
# This is the value that is being used to calculate distance from all the vectors 
def KNN_algorithm(Xtrain_dataset,movie_picked):
  
   
   result = getKNearestNeighbour(Xtrain_dataset,movie_picked.values,10)

   recommended_movies = pd.DataFrame()


   for i in range(len(result)):
   
   
      testDataSet.loc[(testDataSet['genres'] == result[i][0]) & (testDataSet['budget'] == result[i][1]) & (testDataSet['revenue'] == result[i][2]) & (testDataSet['runtime'] == result[i][3]) & (testDataSet['vote_average'] == result[i][4])]
      recommended_movies = recommended_movies.append(testDataSet.loc[(testDataSet['genres'] == result[i][0]) & (testDataSet['budget'] == result[i][1]) & (testDataSet['revenue'] == result[i][2]) & (testDataSet['runtime'] == result[i][3]) & (testDataSet['vote_average'] == result[i][4])],ignore_index=True)
   recommended_movies = recommended_movies.drop_duplicates(keep='first')
   testDataSet.head()
   # print(recommended_movies.iloc[1:len(recommended_movies)].title)

   newLabel = recommended_movies['labels'].max()
   return recommended_movies,newLabel

In [77]:
def newMoive():
    print('Enter the genre of the movie')
    genre = input()
    genre=stringToNumber(genre)
    genre = float(genre)

    print('Enter the budget of the movie')

    budget = float(input())

    print('Enter the runtime of the movie')
    runtime = float(input())

    print('Enter the revenue of the movie')
    revenue = float(input())

    print('Enter the vote of the movie')
    vote_average = float(input())

    print('Enter the title of the movie')
    moive_title = input()

    new_row = {'genres':genre,'budget':budget,		'revenue':revenue,	'runtime' :runtime	,'vote_average':vote_average}
    return new_row,moive_title
   
  

# New Moive 

In [78]:

new_row , moive_title= newMoive();
tempDataset=tempDataset.append(new_row,ignore_index=True)
tempDataset=((tempDataset - tempDataset.min())/(tempDataset.max() - tempDataset.min()))*9 +1

Xtrain_dataset=Xtrain_dataset.append(tempDataset.iloc[-1],ignore_index=True)
recommended_movies,newLabel = KNN_algorithm(Xtrain_dataset,Xtrain_dataset.iloc[-1])

new_row = {'genres':tempDataset.iloc[-1][0]	,'budget':tempDataset.iloc[-1][1]	,'revenue':tempDataset.iloc[-1][2],'title':moive_title ,	'runtime' :tempDataset.iloc[-1][3]	,'vote_average':tempDataset.iloc[-1][4] ,'labels':newLabel}
testDataSet=testDataSet.append(new_row,ignore_index=True)
print('movie picked:=',testDataSet.iloc[-1].title)

Enter the genre of the movie
Enter the budget of the movie
Enter the runtime of the movie
Enter the revenue of the movie
Enter the vote of the movie
Enter the title of the movie
movie picked:= fighter


In [79]:
testDataSet.iloc[-1]

budget          2.015021
genres              10.0
revenue         1.001515
title            fighter
runtime              5.5
vote_average    8.758621
labels               4.0
Name: 54, dtype: object

# User Watched 

In [80]:

user_watched  = Xtrain_dataset.iloc[40]
print('moive_watched', testDataSet.iloc[20].title)
recommended_movies,newLabel=KNN_algorithm(Xtrain_dataset,user_watched)
for i in range(len(recommended_movies)):
    usreMovie_recommended =usreMovie_recommended.append(recommended_movies.iloc[i])

ourRecommendation =usreMovie_recommended.sample(n=10)
# print(usreMovie_recommended.iloc[1:len(ourRecommendation)].title)
print(ourRecommendation.iloc[1:len(ourRecommendation)].title,'here')

moive_watched The Goods: Live Hard, Sell Hard
8              Mars Attacks!
7                    Blended
6               The Terminal
9    Bringing Down the House
5                   Non-Stop
4        Big Momma's House 2
1          We're the Millers
3             Little Fockers
2       Cheaper by the Dozen
Name: title, dtype: object here
